In [1]:
import numpy as np
from cubic_subproblem_solver import *

# Helper functions

In [2]:
def sigmoid(t: np.array):
    # if t.dtype is integer, computation will fail
    assert t.dtype == np.float64 or t.dtype == np.float32
    mask = t >= 0
    t[mask] = 1 / (1 + np.exp(-t[mask]))
    t[~mask] = 1 - 1 / (1 + np.exp(t[~mask]))
    return t


def train_Cubic_Newton(X, y, w0, n_iters, M_0):
    M = M_0
    w = w0
    for i in range(n_iters):
        loss_ = loss(X,y,w)
        grad = gradient(X,y,w)
        hess = hessian(X, y, w)
        h = cubic_subproblem_solver(grad, hess, M)
        w_next = w + h
        while (loss(X,y,w_next) > quadratic_form(loss_,grad,hess,M,w,w_next)):
            M *= 2
            h = cubic_subproblem_solver(grad, hess, M)
            w_next = w + h
        w = w_next
        M /= 2
        print(f'iter {i+1: 3,d}: loss = {loss(X,y,w): .5f}, M = {M}')
    return w


def train_GD(X, y, w0, n_iters, lr):
    w = w0
    for i in range(n_iters):
        grad = gradient(X,y,w)
        w = w - lr * grad
        print(f'iter {i+1: 4,d}: loss = {loss(X,y,w): .5f}')
    return w

# Regression with cubic loss

$\mathcal{L}(w) = \frac{1}{N} \sum_{i=1}^{N} |y_i - x_i^T w|^3$

In [3]:
# Creating dataset

X = np.random.uniform(-10, 10, (1000, 5))
X = np.insert(X,-1,values=1,axis=1) #add bias
w_true = np.array([-2.73, 3.14, 2.71, -1.12, -4.56, 0.51])
score = X @ w_true
noise = np.random.randn(*score.shape) * 5
y = score + noise

In [4]:
def loss(X, y, w):
    return np.mean(np.abs(y - X@w)**3)

def gradient(X, y, w):
    N = X.shape[0]
    err = y - X@w
    return (-3/N) * X.T @ (err**2 * np.sign(err))

def hessian(X, y, w):
    N = X.shape[0]
    err = y - X@w
    return (6/N) * X.T @ np.diag(err*np.sign(err)) @ X

def quadratic_form(loss_w, grad_w, hess_w, M, w, w_new):
    diff = w_new - w
    return loss_w + np.dot(grad_w,diff) + 0.5 * np.dot(hess_w @ diff, diff) + M/6 * np.linalg.norm(diff,2)**3

In [5]:
w0 = np.zeros(X.shape[1])
M_0 = 0.01
n_iters = 10

w = train_Cubic_Newton(X, y, w0, n_iters, M_0)

iter   1: loss =  5564.58477, M = 0.005
iter   2: loss =  919.95181, M = 0.0025
iter   3: loss =  268.41898, M = 0.00125
iter   4: loss =  180.54423, M = 0.000625
iter   5: loss =  176.92169, M = 0.0003125
iter   6: loss =  176.92102, M = 0.00015625
iter   7: loss =  176.92102, M = 7.8125e-05
iter   8: loss =  176.92102, M = 3.90625e-05
iter   9: loss =  176.92102, M = 87960930222.08
iter  10: loss =  176.92102, M = 87960930222.08


In [6]:
w0 = np.zeros(X.shape[1])
n_iters = 100
lr = 1e-4 # 1e-3 already diverges
w = train_GD(X, y, w0, n_iters, lr)

iter    1: loss =  8208.11353
iter    2: loss =  3904.21992
iter    3: loss =  2363.10757
iter    4: loss =  1633.48935
iter    5: loss =  1231.67801
iter    6: loss =  987.67753
iter    7: loss =  828.91429
iter    8: loss =  720.27715
iter    9: loss =  643.05895
iter   10: loss =  586.52905
iter   11: loss =  544.16046
iter   12: loss =  511.78763
iter   13: loss =  486.64608
iter   14: loss =  466.84208
iter   15: loss =  451.04535
iter   16: loss =  438.30219
iter   17: loss =  427.91671
iter   18: loss =  419.37209
iter   19: loss =  412.27667
iter   20: loss =  406.32990
iter   21: loss =  401.29925
iter   22: loss =  397.00327
iter   23: loss =  393.29953
iter   24: loss =  390.07550
iter   25: loss =  387.24150
iter   26: loss =  384.72570
iter   27: loss =  382.47036
iter   28: loss =  380.42891
iter   29: loss =  378.56362
iter   30: loss =  376.84382
iter   31: loss =  375.24456
iter   32: loss =  373.74549
iter   33: loss =  372.32996
iter   34: loss =  370.98435
iter   35

# Logistic regression

$\mathcal{L}(w) = \frac{1}{N} \sum_{i=1}^N \log \left( 1 + \exp(-y_i x_i^T w) \right)$

In [7]:
# Creating dataset

X = np.random.uniform(-10, 10, (1000, 5))
w_true = np.array([3.14, 2.71, -1.12, -4.56, 0.51])
score = X @ w_true
noise = np.random.randn(*score.shape) * 25
y = np.sign(score + noise)

c1 = y[y == 1].size
c2 = y[y == -1].size
print(f'{c1} samples of positive class and {c2} samples of negative class')

490 samples of positive class and 510 samples of negative class


In [8]:
def loss(X, y, w):
    return np.mean(- np.log(sigmoid(y * (X@w))))

def gradient(X, y, w):
    N = X.shape[0]
    return (1/N) * X.T @ (-y * sigmoid(-y * (X@w)))

def hessian(X, y, w):
    N = X.shape[0]
    probs = sigmoid(-y*(X@w))
    return (1/N) * X.T @ np.diag(probs*(1-probs)) @ X

def quadratic_form(loss_w, grad_w, hess_w, M, w, w_new):
    diff = w_new - w
    return loss_w + np.dot(grad_w,diff) + 0.5 * np.dot(hess_w @ diff, diff) + M/6 * np.linalg.norm(diff,2)**3

In [9]:
w0 = np.zeros(X.shape[1])
M_0 = 0.01
n_iters = 10

w = train_Cubic_Newton(X, y, w0, n_iters, M_0)

iter   1: loss =  0.43857, M = 0.005
iter   2: loss =  0.40460, M = 0.0025
iter   3: loss =  0.40068, M = 0.00125
iter   4: loss =  0.40060, M = 0.000625
iter   5: loss =  0.40060, M = 0.0003125
iter   6: loss =  0.40060, M = 0.005
iter   7: loss =  0.40060, M = 0.0025
iter   8: loss =  0.40060, M = 0.00125
iter   9: loss =  0.40060, M = 0.000625
iter  10: loss =  0.40060, M = 0.0003125


In [10]:
w0 = np.zeros(X.shape[1])
n_iters = 50
lr = 0.1
w = train_GD(X, y, w0, n_iters, lr)

iter    1: loss =  0.45788
iter    2: loss =  0.42787
iter    3: loss =  0.41595
iter    4: loss =  0.40996
iter    5: loss =  0.40658
iter    6: loss =  0.40454
iter    7: loss =  0.40326
iter    8: loss =  0.40242
iter    9: loss =  0.40186
iter   10: loss =  0.40148
iter   11: loss =  0.40122
iter   12: loss =  0.40104
iter   13: loss =  0.40091
iter   14: loss =  0.40082
iter   15: loss =  0.40076
iter   16: loss =  0.40071
iter   17: loss =  0.40068
iter   18: loss =  0.40066
iter   19: loss =  0.40064
iter   20: loss =  0.40063
iter   21: loss =  0.40062
iter   22: loss =  0.40061
iter   23: loss =  0.40061
iter   24: loss =  0.40061
iter   25: loss =  0.40060
iter   26: loss =  0.40060
iter   27: loss =  0.40060
iter   28: loss =  0.40060
iter   29: loss =  0.40060
iter   30: loss =  0.40060
iter   31: loss =  0.40060
iter   32: loss =  0.40060
iter   33: loss =  0.40060
iter   34: loss =  0.40060
iter   35: loss =  0.40060
iter   36: loss =  0.40060
iter   37: loss =  0.40060
i